<a href="https://colab.research.google.com/github/law-san-667/angular-snapface-app/blob/main/projet_malware_OumarDiop%26LawalDanAzoumi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**1. Importation du Dataset:**

In [8]:
# Import des bibliothèques nécessaires
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report

In [9]:
#Importation du dataset
dt = pd.read_csv("/content/sample_data/DatasetmalwareExtrait.csv")

**2. Séparer le dataset en features (caractéristiques) et label (étiquettes).**

In [10]:
#Séparation des features
X = dt.drop(columns='legitimate')
y = dt['legitimate']

**3. Division des Données:**

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42) # 30% pour le test

**4 . Choix des Algorithmes:**

C'est une partie importante, les 4 algorithmes que nous avons choisi pour notre situation sont les suivant:

* **Forêt Aléatoire :**

  * Avantages : Robuste, résiste au surajustement (overfitting), capable de gérer des ensembles de données complexes.
  * Inconvénients : Peut être gourmand en ressources, peut être difficile à interpréter.

* **Arbre de Décision :**

  * Avantages : Interprétable, peu de préparation des données nécessaire, peut gérer des données catégorielles.
  * Inconvénients : Tendance au surajustement, sensibilité aux variations mineures des données d'entrée.

* **k-plus proches voisins (KNN) :**

  * Avantages : Facile à comprendre et à implémenter, adapté aux données non linéaires, ne nécessite pas d'entraînement.
  * Inconvénients : Sensible à la dimensionnalité (mauvaise performance avec de nombreuses caractéristiques), peut être lent pour des ensembles de données volumineux.

* **Réseau de Neurones :**

  * Avantages : Capable de capturer des relations complexes, performant sur de grandes quantités de données, peut apprendre des représentations hiérarchiques.
  * Inconvénients : Exige un grand ensemble de données pour éviter le surajustement, peut être difficile à interpréter, nécessite des ressources de calcul importantes, peut nécessiter un temps d'entraînement prolongé.

**5. Entraînement des Modèles:**

In [12]:
# Modèle Forêt Aléatoire
rf_classifier = RandomForestClassifier()
rf_classifier.fit(X_train, y_train)

# Modèle Arbre de Decision
dt_classifier = DecisionTreeClassifier()
dt_classifier.fit(X_train, y_train)

# Modèle k-plus proches voisins (KNN)
knn_classifier = KNeighborsClassifier()
knn_classifier.fit(X_train, y_train)

# Modèle Réseau de Neurones
mlp_classifier = MLPClassifier()
mlp_classifier.fit(X_train, y_train)

MLPClassifier()

**6. Tester la performance de chaque modèle**

In [13]:
# Modèle Forêt Aléatoire
rf_pred = rf_classifier.predict(X_test)

# Modèle Arbre de Decision
dt_pred = dt_classifier.predict(X_test)

# Modèle k-plus proches voisins (KNN)
knn_pred = knn_classifier.predict(X_test)

# Modèle Réseau de Neurones
mlp_pred = mlp_classifier.predict(X_test)

**7. Évaluer les Modèles: **

In [14]:


# Évaluation des modèles
print("\nMLP Accuracy:", accuracy_score(y_test, mlp_pred))
print("MLP Classification Report:\n", classification_report(y_test, mlp_pred))

print("\nRandom Forest Accuracy:", accuracy_score(y_test, rf_pred))
print("Random Forest Classification Report:\n", classification_report(y_test, rf_pred))

print("\nDecision Tree Accuracy:", accuracy_score(y_test, dt_pred))
print("Decision Tree Classification Report:\n", classification_report(y_test, dt_pred))

print("\nKNN Accuracy:", accuracy_score(y_test, knn_pred))
print("KNN Classification Report:\n", classification_report(y_test, knn_pred))




MLP Accuracy: 0.8891206286074599
MLP Classification Report:
               precision    recall  f1-score   support

           0       0.90      0.95      0.92     28884
           1       0.87      0.74      0.80     12350

    accuracy                           0.89     41234
   macro avg       0.88      0.85      0.86     41234
weighted avg       0.89      0.89      0.89     41234


Random Forest Accuracy: 0.990857059708008
Random Forest Classification Report:
               precision    recall  f1-score   support

           0       0.99      0.99      0.99     28884
           1       0.98      0.99      0.98     12350

    accuracy                           0.99     41234
   macro avg       0.99      0.99      0.99     41234
weighted avg       0.99      0.99      0.99     41234


Decision Tree Accuracy: 0.9875103070281807
Decision Tree Classification Report:
               precision    recall  f1-score   support

           0       0.99      0.99      0.99     28884
           1

**8. Sélection du Meilleur Modèle:**
**Le model RandomForest est donc plus efficace avec une presicion de 99,1%**
C'est donc celui que nous allons enregistrer...

**9. Identifier et enregistrer le modèle présentant la meilleure précision.**

In [15]:
import joblib
joblib.dump(rf_classifier, 'randomForestClassifier_model.pkl')

['randomForestClassifier_model.pkl']

Le modèle a été sauvegardé !

**10. Le déploiement:** Utilisons Flask



In [17]:
from flask import Flask, request, jsonify, render_template

app = Flask(__name__)

model = joblib.load('randomForestClassifier_model.pkl')
predictions = model.predict(X_test)

@app.route("/")
def home():
  return render_template("index.html")


@app.route("/model", methods=['POST'])
def predict():
  try:
    data = request.json
    prediction = model.predict([data["features"]])[0]
    return render_template("/prediction.html", result=prediction)

  except Exception as error:
    return "Something wrong happenned"

  if __main__ == "__main__":
    app.run(port=5000, debug=True)